Lorenz performance experiments

In [1]:
import numpy as np
import torch
import sys

# import matplotlib.pyplot as plt

sys.path.append("../")
from utils import simulate_lorenz_96, compute_spline  # , compare_graphs
import utils as utils
import NMC as models
import importlib

## SVAM

In [2]:
# LiNGAM / SVAM performance with sparse data
import warnings

warnings.filterwarnings("ignore")
for dt in [0.01, 0.05, 0.1, 0.15, 0.25]:
    perf = []
    for i in range(20):
        # Simulate data
        p = 10
        T = 1000
        num_points = T
        data, GC = simulate_lorenz_96(p, T=T, delta_t=dt, sigma=0.05, F=10)

        # format for NeuralODE
        times = np.linspace(0, T, num_points)
        times_np = np.hstack([times[:, None]])
        times = torch.from_numpy(times_np[:, :, None])
        data = torch.from_numpy(data[:, None, :].astype(np.float32))

        indices = np.random.choice(range(T), int(T), replace=False)
        indices = np.sort(indices)

        from benchmarks.lingam_benchmark import lingam_method

        importlib.reload(utils)
        graph = lingam_method(data[indices].squeeze().detach())
        perf.append(utils.compare_graphs(GC, graph))  # tpr, fdr

    print("Means and standard deviations for TPR, FDR and AUC with", dt, "time interval")
    print(np.mean(np.reshape(perf, (-1, 3)), axis=0), np.std(np.reshape(perf, (-1, 3)), axis=0))

Means and standard deviations for TPR, FDR and AUC with 0.01 time interval
[0.3975     0.32996842 0.69335417] [0.04175823 0.06499461 0.02458924]
Means and standard deviations for TPR, FDR and AUC with 0.05 time interval
[0.45875    0.20603195 0.70964583] [0.03814037 0.07344687 0.02074911]
Means and standard deviations for TPR, FDR and AUC with 0.1 time interval
[0.58625    0.14463012 0.6938125 ] [0.03208874 0.04376159 0.01608032]
Means and standard deviations for TPR, FDR and AUC with 0.15 time interval
[0.66125    0.22614872 0.64844792] [0.04839615 0.04318911 0.02568949]
Means and standard deviations for TPR, FDR and AUC with 0.25 time interval
[0.79625    0.49919492 0.54761458] [0.05492893 0.02473709 0.0136025 ]


In [3]:
# LiNGAM / SVAM performance with irregular data
for fraction in [1, 0.7, 0.5, 0.3]:
    perf = []
    for i in range(20):
        # Simulate data
        p = 10
        T = 1000
        num_points = T
        data, GC = simulate_lorenz_96(p, T=T, delta_t=0.1, sigma=0.05, F=10)

        # format for NeuralODE
        times = np.linspace(0, T, num_points)
        times_np = np.hstack([times[:, None]])
        # times = torch.from_numpy(times_np[:, :, None])
        data = torch.from_numpy(data[:, None, :].astype(np.float32))

        indices = np.random.choice(range(T), int(fraction * T), replace=False)
        indices = np.sort(indices)

        data = compute_spline(data[indices].squeeze().detach(), times[indices], k=2, s=0)

        from benchmarks.lingam_benchmark import lingam_method

        importlib.reload(utils)
        graph = lingam_method(data)
        perf.append(utils.compare_graphs(GC, graph))  # tpr, fdr

    print("Means and standard deviations for TPR, FDR and AUC with", fraction, "kept")
    print(np.mean(np.reshape(perf, (-1, 3)), axis=0), np.std(np.reshape(perf, (-1, 3)), axis=0))

Means and standard deviations for TPR, FDR and AUC with 1 kept
[0.57375    0.14827103 0.70214583] [0.03304826 0.05514702 0.01647627]
Means and standard deviations for TPR, FDR and AUC with 0.7 kept
[0.64875    0.27761763 0.64661458] [0.04967079 0.05612829 0.02489761]
Means and standard deviations for TPR, FDR and AUC with 0.5 kept
[0.66       0.42124403 0.59220833] [0.05937171 0.05131919 0.0254867 ]
Means and standard deviations for TPR, FDR and AUC with 0.3 kept
[0.57125    0.56894655 0.53840625] [0.05435704 0.04234778 0.03246341]


# DCM

In [4]:
# DCM performance with sparse data
for dt in [0.01, 0.05, 0.1, 0.25, 0.5, 1]:
    perf = []
    for i in range(10):
        # Simulate data
        p = 10
        T = 1000
        num_points = T
        data, GC = simulate_lorenz_96(p, T=T, delta_t=dt, sigma=0.01, F=10)

        indices = np.random.choice(range(T), int(T), replace=False)
        indices = np.sort(indices)

        from benchmarks.DCM import DCM_full

        graph = DCM_full(data[indices], lambda1=0.05, s=4, w_threshold=0.1)
        # plt.matshow(abs(graph),cmap='Reds')
        # plt.colorbar()
        # plt.show()
        perf.append(utils.compare_graphs(GC, graph))  # tpr, fdr

    print("Means and standard deviations for TPR, FDR and AUC with", dt, "time interval")
    print(np.mean(np.reshape(perf, (-1, 3)), axis=0), np.std(np.reshape(perf, (-1, 3)), axis=0))

Means and standard deviations for TPR, FDR and AUC with 0.01 time interval
[0.005 0.    0.503] [0.01  0.    0.005]
Means and standard deviations for TPR, FDR and AUC with 0.05 time interval
[0.753 0.058 0.872] [0.008 0.034 0.002]
Means and standard deviations for TPR, FDR and AUC with 0.1 time interval
[0.938 0.161 0.958] [0.036 0.026 0.02 ]
Means and standard deviations for TPR, FDR and AUC with 0.25 time interval
[1.    0.6   0.936] [0.000e+00 1.110e-16 2.831e-02]
Means and standard deviations for TPR, FDR and AUC with 0.5 time interval
[1.    0.6   0.588] [0.000e+00 1.110e-16 3.613e-02]
Means and standard deviations for TPR, FDR and AUC with 1 time interval
[1.    0.6   0.508] [0.000e+00 1.110e-16 4.837e-02]


In [5]:
# DCM performance with irregular data
for fraction in [1, 0.7, 0.5, 0.3]:
    perf = []
    for i in range(10):
        # Simulate data
        p = 10
        T = 1000
        num_points = T
        data, GC = simulate_lorenz_96(p, T=T, delta_t=0.05, sigma=0.00, F=10)

        indices = np.random.choice(range(T), int(fraction * T), replace=False)
        indices = np.sort(indices)

        # data = compute_spline(data[indices].squeeze().detach(), times[indices], k=2,s=0)

        from benchmarks.DCM import DCM_full

        graph = DCM_full(data[indices], lambda1=0.05, s=4, w_threshold=0.1)
        perf.append(utils.compare_graphs(GC, graph))  # tpr, fdr

    print("Means and standard deviations for TPR, FDR and AUC with", fraction, "kept")
    print(np.mean(np.reshape(perf, (-1, 3)), axis=0), np.std(np.reshape(perf, (-1, 3)), axis=0))

Means and standard deviations for TPR, FDR and AUC with 1 kept
[0.75  0.081 0.869] [0.    0.034 0.003]
Means and standard deviations for TPR, FDR and AUC with 0.7 kept
[0.82  0.131 0.899] [0.048 0.041 0.024]
Means and standard deviations for TPR, FDR and AUC with 0.5 kept
[0.92  0.322 0.926] [0.029 0.063 0.021]
Means and standard deviations for TPR, FDR and AUC with 0.3 kept
[1.    0.596 0.806] [0.    0.005 0.047]


# PCMCI

In [4]:
# pcmci performance with sparse data
for dt in [0.01, 0.05, 0.1, 0.15, 0.25]:
    perf = []
    for i in range(10):
        # Simulate data
        p = 10
        T = 1000
        num_points = T
        data, GC = simulate_lorenz_96(p, T=T, delta_t=dt, sigma=0.05, F=10)

        # format for NeuralODE
        # times = np.linspace(0, T, num_points)
        # times_np = np.hstack([times[:, None]])
        # times = torch.from_numpy(times_np[:, :, None])
        # data = torch.from_numpy(data[:, None, :].astype(np.float32))

        indices = np.random.choice(range(T), int(T), replace=False)
        indices = np.sort(indices)

        from benchmarks.pcmci import pcmci

        importlib.reload(utils)
        graph = pcmci(data[indices])
        perf.append(utils.compare_graphs(GC, graph))  # tpr, fdr

    print("Means and standard deviations for TPR, FDR and AUC with", dt, "time interval")
    print(np.mean(np.reshape(perf, (-1, 3)), axis=0), np.std(np.reshape(perf, (-1, 3)), axis=0))

Could not import packages for CMIknn and GPDC estimation
Could not import r-package RCIT
Means and standard deviations for TPR, FDR and AUC with 0.01 time interval
[0.255      0.03560606 0.62416667] [0.01       0.04365767 0.00612372]
Means and standard deviations for TPR, FDR and AUC with 0.05 time interval
[0.31       0.07527681 0.64583333] [0.03905125 0.06180708 0.01844662]
Means and standard deviations for TPR, FDR and AUC with 0.1 time interval
[0.6475    0.0821358 0.80375  ] [0.0675     0.03210886 0.02913391]
Means and standard deviations for TPR, FDR and AUC with 0.15 time interval
[0.67       0.01470184 0.83166667] [0.035      0.01802885 0.01920286]
Means and standard deviations for TPR, FDR and AUC with 0.25 time interval
[0.27       0.4780816  0.55416667] [0.06103278 0.0707631  0.03405469]


In [5]:
# pcmci performance with irregular data
for fraction in [1, 0.7, 0.5, 0.3]:
    perf = []
    for i in range(10):
        # Simulate data
        p = 10
        T = 1000
        num_points = T
        data, GC = simulate_lorenz_96(p, T=T, delta_t=0.1, sigma=0.05, F=10)

        # format for NeuralODE
        times = np.linspace(0, T, num_points)
        times_np = np.hstack([times[:, None]])
        # times = torch.from_numpy(times_np[:, :, None])
        data = torch.from_numpy(data[:, None, :].astype(np.float32))

        indices = np.random.choice(range(T), int(fraction * T), replace=False)
        indices = np.sort(indices)

        data = compute_spline(data[indices].squeeze().detach(), times[indices], k=2, s=0)

        from benchmarks.pcmci import pcmci

        importlib.reload(utils)
        graph = pcmci(data)
        perf.append(utils.compare_graphs(GC, graph))  # tpr, fdr

    print("Means and standard deviations for TPR, FDR and AUC with", fraction, "kept")
    print(np.mean(np.reshape(perf, (-1, 3)), axis=0), np.std(np.reshape(perf, (-1, 3)), axis=0))

Means and standard deviations for TPR, FDR and AUC with 1 kept
[0.6325    0.1049527 0.79125  ] [0.10491068 0.04194103 0.05049924]
Means and standard deviations for TPR, FDR and AUC with 0.7 kept
[0.3825  0.      0.69125] [0.06025985 0.         0.03012993]
Means and standard deviations for TPR, FDR and AUC with 0.5 kept
[0.055      0.01666667 0.52666667] [0.03316625 0.05       0.01493039]
Means and standard deviations for TPR, FDR and AUC with 0.3 kept
[0.         0.1        0.49916667] [0.     0.3    0.0025]


## NGM

In [8]:
# NGM performance with sparse data
import warnings

warnings.filterwarnings("ignore")
for dt in [0.01, 0.05, 0.1, 0.15, 0.25]:
    perf = []
    for i in range(10):
        # Simulate data
        p = 10
        T = 1000
        num_points = T
        data, GC = simulate_lorenz_96(p, T=T, delta_t=dt, sigma=0.05, F=10)

        # format for NeuralODE
        times = np.linspace(0, T, num_points)
        times_np = np.hstack([times[:, None]])
        times = torch.from_numpy(times_np[:, :, None])  # .astype(np.float32))
        data = torch.from_numpy(data[:, None, :])  # .astype(np.float32))

        import NMC as models

        func = models.MLPODEF(dims=[p, 12, 1], GL_reg=0.1)

        # GL training
        models.train(func, data, n_steps=1000, plot=False, plot_freq=20)
        # AGL training
        weights = func.group_weights()
        func.GL_reg *= 1 / weights
        func.reset_parameters()
        models.train(func, data, n_steps=1000, plot=False, plot_freq=20)
        graph = func.causal_graph(w_threshold=0.1)
        perf.append(utils.compare_graphs(GC, graph))  # tpr, fdr

    print("Means and standard deviations for TPR, FDR and AUC with", dt, "time interval")
    print(np.mean(np.reshape(perf, (-1, 3)), axis=0), np.std(np.reshape(perf, (-1, 3)), axis=0))

Means and standard deviations for TPR, FDR and AUC with 0.01 time interval
[0.633 0.    0.816] [0.091 0.    0.045]
Means and standard deviations for TPR, FDR and AUC with 0.05 time interval
[0.75  0.    0.875] [0. 0. 0.]
Means and standard deviations for TPR, FDR and AUC with 0.1 time interval
[0.81  0.    0.905] [0.025 0.    0.013]
Means and standard deviations for TPR, FDR and AUC with 0.15 time interval
[0.993 0.    0.996] [0.011 0.    0.006]
Means and standard deviations for TPR, FDR and AUC with 0.25 time interval
[0.99  0.164 0.994] [0.012 0.056 0.006]


In [11]:
# NGM performance with irregular data
for fraction in [1, 0.7, 0.5, 0.3]:
    perf = []
    for i in range(10):
        # Simulate data
        p = 10
        T = 3000
        num_points = T
        data, GC = simulate_lorenz_96(p, T=T, delta_t=0.05, sigma=0.05, F=5)

        # format for NeuralODE
        indices = np.random.choice(range(T), int(fraction * T), replace=False)
        indices = np.sort(indices)
        times = np.linspace(0, T, num_points)
        times_np = np.hstack([times[indices, None]])
        data = data[indices, :]
        times = torch.from_numpy(times_np[:, :, None])
        data = torch.from_numpy(data[:, None, :])  # .astype(np.float32))

        import NMC as models

        func = models.MLPODEF(dims=[p, 12, 1], GL_reg=0.05)
        models.train(func, data, n_steps=2000, horizon=5, plot_freq=20, plot=False, irregular=True, times=times)
        # weights = func.group_weights()
        # func.GL_reg *= (1 / weights)
        # func.reset_parameters()
        # models.train(func,data,n_steps=1000,horizon = 5, plot_freq=20, plot=False, irregular=True, times=times)
        graph = func.causal_graph(w_threshold=0.09)
        # print(graph)
        perf.append(utils.compare_graphs(GC, graph))  # tpr, fdr

    print("Means and standard deviations for TPR, FDR and AUC with", fraction, "kept")
    print(np.mean(np.reshape(perf, (-1, 3)), axis=0), np.std(np.reshape(perf, (-1, 3)), axis=0))

Means and standard deviations for TPR, FDR and AUC with 1 kept
[0.985 0.078 0.989] [0.017 0.05  0.008]
Means and standard deviations for TPR, FDR and AUC with 0.7 kept
[0.982 0.213 0.985] [0.03  0.044 0.02 ]
Means and standard deviations for TPR, FDR and AUC with 0.5 kept
[0.915 0.214 0.938] [0.139 0.051 0.082]
Means and standard deviations for TPR, FDR and AUC with 0.3 kept
[0.605 0.262 0.767] [0.293 0.14  0.148]
